In [5]:
# limpio la memoria
Sys.time()
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

[1] "2025-12-04 01:14:47 UTC"

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,836511,44.7,1454648,77.7,1454648,77.7
Vcells,1601617,12.3,1185329759,9043.4,1477482120,11272.3


In [6]:
path_competencia_02 <- '/home/vickydiliscia/buckets/b1/datasets/competencia_02_crudo.csv.gz'
path_competencia_03 <- '/home/vickydiliscia/buckets/b1/datasets/competencia_03_crudo.csv.gz'
path_functions <- '/home/vickydiliscia/dmeyf2025/src/churn/functions'

In [7]:
source(file.path(path_functions, "index.R"))
source_functions(path_functions)

Sourcing: add_divisiones.R

Sourcing: add_ratio.R

Sourcing: agregar_clase_ternaria.R

Sourcing: agregar_var_rf.R

Sourcing: aplicar_dataquality_zeros.R

Sourcing: apply_sum_specs_multi.R

Sourcing: audit_nullear.R

Sourcing: build_variables_rotas.R

Sourcing: crear_proporcion.R

Sourcing: deflactar_por_ipc.R

Sourcing: features_historicos_optimizados.R

Sourcing: features_historicos.R

Sourcing: flip.R

Sourcing: interpolar_faltantes.R

Sourcing: lags.R

Sourcing: normalizar_ctrx_quarter.R

Sourcing: print_utils.R

Sourcing: run_features_historicas.R

Sourcing: tendencias_y_mas.R

Sourcing: ver_meses_rotos.R

✅ Todas las funciones fueron cargadas correctamente.



In [8]:
paths <- c(path_competencia_02,
           path_competencia_03)

dataset <- rbindlist(
  lapply(paths, fread),
  use.names = TRUE,
  fill      = TRUE
)

dataset <- agregar_clase_ternaria(dataset)
setorder(dataset, foto_mes, clase_ternaria, numero_de_cliente)
dataset[, .N, .(foto_mes, clase_ternaria)]
Sys.time()

foto_mes,clase_ternaria,N
<int>,<chr>,<int>
201901,BAJA+1,645
201901,BAJA+2,729
201901,CONTINUA,122899
201902,BAJA+1,733
201902,BAJA+2,707
201902,CONTINUA,123961
201903,BAJA+1,708
201903,BAJA+2,751
201903,CONTINUA,124508


[1] "2025-12-04 01:15:07 UTC"

In [9]:
library(data.table)

#--------------------------------------------------------
# 1) Clientes de 202109
#--------------------------------------------------------

df_202109 <- dataset[ foto_mes == 202109, .(numero_de_cliente, foto_mes) ]

cat("Clientes en 202109:", nrow(df_202109), "\n")

#--------------------------------------------------------
# 2) Leer TODAS las predicciones y ensamblar simple
#--------------------------------------------------------

dirs_preds <- c(
  "/home/vickydiliscia/buckets/b1/exp/APO3012-202107_100271/predicciones_produccion",  # ej: "predicciones_modelitos_APO1"
  "/home/vickydiliscia/buckets/b1/exp/APO3012-202107_250061/predicciones_produccion",  # ej: "predicciones_modelitos_APO2"
  "/home/vickydiliscia/buckets/b1/exp/APO3002-202107_100271/predicciones_produccion",
  "/home/vickydiliscia/buckets/b1/exp/APO3002-202107_250061/predicciones_produccion"
)


Clientes en 202109: 165093 


In [10]:
# Archivos dentro de todas las carpetas
archivos <- unlist(lapply(dirs_preds, function(d) {
  list.files(d, pattern = "\\.(txt|csv)$", full.names = TRUE)
}))

cat("Cantidad de archivos de predicción encontrados:", length(archivos), "\n")

# Leer y normalizar: numero_de_cliente + prob
lista_preds <- lapply(archivos, function(f) {
  dt <- fread(f)

  # numero_de_cliente debe existir
  if (!"numero_de_cliente" %in% names(dt)) {
    stop("ERROR: el archivo ", f, " no contiene numero_de_cliente")
  }

  # prob debe existir
  if (!"prob" %in% names(dt)) {
    stop("ERROR: el archivo ", f, " no contiene la columna 'prob'")
  }

  dt[, .(numero_de_cliente, prob)]
})

# Ensamble simple: promedio por cliente
preds_ensamble <- rbindlist(lista_preds)[
  , .(prob = mean(prob, na.rm = TRUE)), by = numero_de_cliente]

cat("Clientes con predicciones ensambladas:", nrow(preds_ensamble), "\n")

#--------------------------------------------------------
# 3) Filtrar clientes del mes 202109
#--------------------------------------------------------

setkey(df_202109,       numero_de_cliente)
setkey(preds_ensamble,  numero_de_cliente)

df_ens_202109 <- df_202109[preds_ensamble, nomatch = 0]

cat("Clientes de 202109 con predicción:", nrow(df_ens_202109), "\n")

#--------------------------------------------------------
# 4) Ordenar por probabilidad y generar submit top 11000
#--------------------------------------------------------

setorder(df_ens_202109, -prob)

N <- 11000L
N <- min(N, nrow(df_ens_202109))

cat("Clientes usados para submit:", N, "\n")


Cantidad de archivos de predicción encontrados: 200 
Clientes con predicciones ensambladas: 165093 
Clientes de 202109 con predicción: 165093 
Clientes usados para submit: 11000 


In [11]:
setwd('/home/vickydiliscia/buckets/b1/exp')

In [12]:
dir.create('Ensamble_C3002_C3012-202109')
setwd('/home/vickydiliscia/buckets/b1/exp/Ensamble_C3002_C3012-202109')

In [13]:
fwrite(
  df_ens_202109[1:N, .(numero_de_cliente)],
  file = "submit_ensamble_202109_C3002-C3012_top11000.csv",
  col.names = FALSE
)

cat("\nArchivo creado: submit_ensamble_202109_top11000.csv\n")


Archivo creado: submit_ensamble_202109_top11000.csv
